In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop

import matplotlib.pyplot as plt

import sys

import numpy as np
import phylo as ph 

Using TensorFlow backend.


In [ ]:
class GAN(object): 
    
    def __init__(self): 
        self.shape = ph.M ** 2 
        self.discriminator = None # Discriminator 
        self.generator = None # Shareholder value generator
        self.adversary_model = None # Adversarial Model 
        self.discriminator_model = None # Discriminator Model 
    
    def discriminate(self): 
        '''
        Discriminator that transforms 96*96 flat list to a source of truth 
        '''
        if self.discriminator: 
            return self.discriminator
        
        self.discriminator = Sequential() 
        depth = 64 
        dropout = 0.4 
        channel = 1
        
        dim = int(self.shape ** .5)
        
        # In: 96*96 flat list 
        # Out: 96 x 96 x 1 
        self.discriminator.add(Dense(dim*dim*channel, input_dim=self.shape))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        # In: dim*dim*channel 
        # Out: 48 x 48 x 1, depth = 64
        self.discriminator.add(Conv2D(depth*1, 5, strides=2, padding='same'))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        self.discriminator.add(Conv2D(depth*2, 5, strides=2, padding='same'))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        self.discriminator.add(Conv2D(depth*4, 5, strides=2, padding='same'))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        self.discriminator.add(Conv2D(depth*8, 5, strides=2, padding='same'))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        self.discriminator.add(Conv2D(depth*16, 5, strides=2, padding='same'))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        self.discriminator.add(Conv2D(depth*32, 5, strides=1, padding='same'))
        self.discriminator.add(LeakyReLU(alpha=0.2))
        self.discriminator.add(Dropout(dropout))
        
        # Flatten back out 
        self.discriminator.add(Flatten())
        self.discriminator.add(Dense(1))
        self.discriminator.add(Activation('sigmoid'))
        self.discriminator.summary()
        return self.discriminator

    def generate(self): 
        '''
        Generator that takes random noise and produces an image from it 
        '''
        if self.generator: 
            return self.generator 
        
        self.generator = Sequential() 
        dropout = 0.4 
        depth = 64 * 16 
        dim = 3 
        
        self.generator.add(Dense(dim*dim*depth, input_dim=self.shape))
        self.generator.add(BatchNormalization(momentum=0.9))
        self.generator.add(Activation('relu'))
        self.generator.add(Reshape((dim, dim, depth)))
        self.generator.add(Dropout(dropout))
        
        self.generator.add(UpSampling2D())
        self.generator.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9))
        self.generator.add(Activation('relu'))
        
        self.generator.add(UpSampling2D())
        self.generator.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9))
        self.generator.add(Activation('relu'))
        
        self.generator.add(UpSampling2D())
        self.generator.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9))
        self.generator.add(Activation('relu'))
        
        self.generator.add(UpSampling2D())
        self.generator.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9))
        self.generator.add(Activation('relu'))
        
        self.generator.add(UpSampling2D())
        self.generator.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
        self.generator.add(BatchNormalization(momentum=0.9))
        self.generator.add(Activation('relu'))
        
        # Get back to 96*96 Flat list 
        self.generator.add(Dense(self.shape))
        self.generator.add(Activation('sigmoid'))
        self.generator.summary()
        return self.generator
    
    def build_dm(self): 
        if self.discriminator_model: 
            return self.discriminator_model 
        
        optimizer = RMSprop(lr=0.0002, decay=6e-8)
        self.discriminator_model = Sequential()
        self.discriminator_model.add(self.discriminate())
        self.discriminator_model.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        
        return self.discriminator_model
    
    def build_am(self): 
        if self.adversary_model: 
            return self.adversary_model 
        
        optimizer = RMSprop(lr=0.0001, decay=3e-8)
        self.adversary_model = Sequential()
        self.adversary_model.add(self.generate())
        self.adversary_model.add(self.discriminate())
        self.adversary_model.compile(loss='binary_crossentropy', optimizer=optimizer,\
            metrics=['accuracy'])
        
        return self.adversary_model

In [ ]:
pokemon = ph.vectorize_pokemon(ph.REGULAR_POKEMON_PATH) + ph.vectorize_pokemon(ph.SHINY_POKEMON_PATH)

Vectorizing pokemon from /Users/rchatrath/workspace/phylo/images/regular/
Vector iteration 0
Vector iteration 100
Vector iteration 200
Vector iteration 300
Vector iteration 400
Vector iteration 500
Vector iteration 600
Done vectorizing
Vectorizing pokemon from /Users/rchatrath/workspace/phylo/images/shiny/
Vector iteration 0


In [ ]:
class Manager(object): 
    
    def __init__(self): 
#         self.pokemon = ph.vectorize_pokemon(ph.REGULAR_POKEMON_PATH) + ph.vectorize_pokemon(ph.SHINY_POKEMON_PATH)
        self.pokemon = pokemon 
        self.GAN = GAN() 
        self.discriminator = self.GAN.build_dm() 
        self.adversarial_model = self.GAN.build_am() 
        self.generator = self.GAN.generate() 
        
    def train(self, train_steps=2000, batch_size=256, save_interval=0): 
        
        for i in range(train_steps): 
            images_train = np.random.choice(self.pokemon, batch_size % len(self.pokemon))
            noise = np.asarray([ph.generate_random() for i in range(batch_size)])
            fake_images = self.generator.predict(noise)
            X = np.concatenate(images_train, fake_images) 
            Y = np.ones([2*batch_size, 1])
            Y[batch_size:, :] = 0
            d_loss = self.discriminator.train_on_batch(X, Y)
            
            Y = np.ones([batch_size, 1])
            noise = np.asarray([ph.generate_random() for i in range(batch_size)])
            a_loss = self.adversarial_model.train_on_batch(noise, y)
            
            log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
            log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
            print(log_mesg)
            
            if save_interval>0:
                if (i+1)%save_interval==0:
                    self.save_image(save2File=True)
            
    def save_image(save2File=False, samples=16, fake=True): 
        if fake: 
            noise = np.asarray([ph.generate_random() for i in range(samples)])
            images = self.generator.predict(noise)
            cnt = 0 
            for img in images: 
                ph.save_image(img, "fake_img_{}.png".format(cnt))
                cnt += 1 

In [ ]:
if __name__ == "__main__": 
    manager = Manager() 
    manager.train(train_steps=5, batch_size=10, save_interval=1)